In [1]:
import pandas as pd

dataset = pd.read_csv('data_folder/dataset_raw.csv',index_col=0)
dataset.isna().sum()
dataset = dataset.dropna()
dataset

,Planet_Mass,Epsilon,Alpha,Stokes,Aspect_Ratio,SigmaSlope,Dust_gap_1,Gas_gap_1,Dust_depth_1,Gas_depth_1,Dust_gap_2,Dust_depth_2,#_DG,#_GG,Unnamed: 15
Sample#,,,,,,,,,,,,,,,
0,950,0.000128,0.0565,0.005240,0.07560,0.0870,0.541,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
1,951,0.000109,0.0352,0.001230,0.02710,0.0880,0.595,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
2,952,0.000226,0.0915,0.003910,0.00137,0.0997,0.662,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
3,953,0.000233,0.0338,0.000607,0.03160,0.0369,0.784,0.554,0.34,6.770000e-09,0.000002,0.275,6.300000e-09,2,1
4,954,0.000174,0.0850,0.004110,0.05350,0.0795,0.739,0.037,0.00,3.570000e-06,0.000000,0.000,0.000000e+00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,495,0.000209,0.0465,0.001940,0.03810,0.0749,0.822,0.788,0.00,1.420000e-07,0.000000,0.000,0.000000e+00,1,0
1176,496,0.000029,0.0238,0.006720,0.05000,0.0585,0.702,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
1177,497,0.000092,0.0947,0.006560,0.03960,0.0384,1.100,0.697,0.00,2.310000e-11,0.000000,0.000,0.000000e+00,1,0


In [23]:
X = dataset.Planet_Mass.values
y = dataset.drop(['Planet_Mass'],axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.15, random_state = 2)
X_train.shape

(1003,)

In [60]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import IsolationForest
model = make_pipeline(StandardScaler(),GradientBoostingRegressor(random_state = 0, n_estimators =700, max_depth = 6, min_samples_split = 2, learning_rate= 0.1))
model.fit(y_train,X_train)
print(model.score(y_train, X_train))

0.8894970756994647


In [61]:
from sklearn.metrics import r2_score

X_pred = model.predict(y_val)
X_pred = X_pred.astype(int)
X_pred

array([ 567,  593,  589,  652,  542,  580,  463,  216,  506,  634,  355,
        143,  621,  413,  581,  525,  715,  521,  863,  680,  685,  603,
        649,  390,  542,  383,  510,  649,  598,  762,  721, 1097, 1005,
        547,  824,  488,  737,  643,  389,   83,  525,  744,  460,  683,
        614,  553,  642,  493,  560,  597,  437,  525,  778,  522,  655,
        700,  395,  730,  106,  659,  694,  567,  627,  484,  517,  848,
        645,  611,  748,  482,  564,  815,  780,  342,  774,  570,  669,
        486,  535,  315,  674,  115,  810,  317,  839,  769,  569,  840,
        697,  661,  650,  677,  579,  600,  116,  438,  674,  750,  747,
        185,  760,  921,  689,  911, 1009,  805,  484,  591,  934,  680,
        674,  466,  723,  542,  789,  566,  298,  665,  582,  192,  509,
        542,  785,  478,  373,  713,  504,  891,  281,  840,  742,  900,
       1076,  405,  891,  538,  490,  567,  685,  607,  875,  453,  795,
        398,  612,  596,  720,  729,  543,  291,  3

In [62]:
df = pd.DataFrame({'Actual':X_val,'Predicted':X_pred})
df



,Actual,Predicted
0,519,567
1,570,593
2,621,589
3,933,652
4,753,542
...,...,...
172,902,438
173,548,445
174,147,568
175,1,548


In [67]:
model = IsolationForest(n_estimators = 700,max_samples = 'auto',contamination = float(0.3),max_features =1.0  )
model.fit(dataset[['Planet_Mass']])

IsolationForest(contamination=0.3, n_estimators=700)

In [69]:
dataset['score'] = model.decision_function(dataset[['Planet_Mass']])
dataset['anomaly'] = model.predict(dataset[['Planet_Mass']])
dataset

,Planet_Mass,Epsilon,Alpha,Stokes,Aspect_Ratio,SigmaSlope,Dust_gap_1,Gas_gap_1,Dust_depth_1,Gas_depth_1,Dust_gap_2,Dust_depth_2,#_DG,#_GG,Unnamed: 15,score,anomaly
Sample#,,,,,,,,,,,,,,,,,
0,950,0.000128,0.0565,0.005240,0.07560,0.0870,0.541,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.008482,1
1,951,0.000109,0.0352,0.001230,0.02710,0.0880,0.595,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.009171,1
2,952,0.000226,0.0915,0.003910,0.00137,0.0997,0.662,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.008265,1
3,953,0.000233,0.0338,0.000607,0.03160,0.0369,0.784,0.554,0.34,6.770000e-09,0.000002,0.275,6.300000e-09,2,1,0.007738,1
4,954,0.000174,0.0850,0.004110,0.05350,0.0795,0.739,0.037,0.00,3.570000e-06,0.000000,0.000,0.000000e+00,1,0,0.006676,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,495,0.000209,0.0465,0.001940,0.03810,0.0749,0.822,0.788,0.00,1.420000e-07,0.000000,0.000,0.000000e+00,1,0,0.026628,1
1176,496,0.000029,0.0238,0.006720,0.05000,0.0585,0.702,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.027167,1
1177,497,0.000092,0.0947,0.006560,0.03960,0.0384,1.100,0.697,0.00,2.310000e-11,0.000000,0.000,0.000000e+00,1,0,0.027007,1


In [70]:
anomaly = dataset.loc[dataset['anomaly']==-1]
anomaly_index = list(anomaly.index)
print(anomaly)

         Planet_Mass   Epsilon   Alpha    Stokes  Aspect_Ratio  SigmaSlope  \
Sample#                                                                      
25              1075  0.000323  0.0727  0.009130       0.09620      0.0639   
26              1076  0.000247  0.0693  0.001590       0.00442      0.0838   
27              1077  0.000036  0.0177  0.000574       0.07220      0.0818   
28              1078  0.000287  0.0457  0.007980       0.09050      0.0408   
29              1079  0.000175  0.0938  0.007180       0.04060      0.0431   
...              ...       ...     ...       ...           ...         ...   
1100            1195  0.000157  0.0769  0.003850       0.00533      0.0728   
1101            1196  0.000320  0.0324  0.000995       0.00748      0.0450   
1102            1197  0.000038  0.0934  0.009450       0.06230      0.0962   
1103            1198  0.000245  0.0949  0.006680       0.09690      0.0637   
1104            1199  0.000323  0.0952  0.006950       0.04850  

In [79]:
outlier_counter = len(dataset[dataset['Dust_gap_1']<0.8])
outlier_counter

353